In [3]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Credit Score Prediction Project\n",
    "\n",
    "In this notebook, we will:\n",
    "- Load and clean the data\n",
    "- Train baseline & tuned XGBoost models\n",
    "- Evaluate with classification metrics\n",
    "- Visualize results (confusion matrix, feature importance)\n",
    "- Use SHAP for explainability\n",
    "- Generate predictions for submission"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Import Libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split, RandomizedSearchCV\n",
    "from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay\n",
    "import xgboost as xgb\n",
    "import shap\n",
    "import joblib\n",
    "import warnings\n",
    "warnings.filterwarnings(\"ignore\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load and Prepare Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load training data\n",
    "train = pd.read_csv(\"train.csv\").sample(10000, random_state=42)\n",
    "\n",
    "# Separate features & target\n",
    "X = train.drop([\"Credit_Score\"], axis=1)\n",
    "y = train[\"Credit_Score\"]\n",
    "\n",
    "# Drop non-informative columns\n",
    "X = X.drop([\"ID\", \"Customer_ID\", \"Name\", \"SSN\"], axis=1)\n",
    "\n",
    "# Handle missing values\n",
    "X = X.fillna(X.mean(numeric_only=True))\n",
    "\n",
    "# Convert categorical variables\n",
    "X = pd.get_dummies(X, drop_first=True)\n",
    "\n",
    "# Encode labels as numbers\n",
    "label_map = {label: idx for idx, label in enumerate(y.unique())}\n",
    "y = y.map(label_map)\n",
    "\n",
    "# Train-test split\n",
    "X_train, X_val, y_train, y_val = train_test_split(\n",
    "    X, y, test_size=0.2, random_state=42, stratify=y\n",
    ")\n",
    "\n",
    "X_train.shape, X_val.shape"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Train XGBoost Model (Tuned)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "xgb_model = xgb.XGBClassifier(\n",
    "    objective=\"multi:softmax\",\n",
    "    num_class=3,\n",
    "    random_state=42,\n",
    "    subsample=0.8,\n",
    "    n_estimators=300,\n",
    "    max_depth=4,\n",
    "    learning_rate=0.05,\n",
    "    colsample_bytree=0.8\n",
    ")\n",
    "\n",
    "xgb_model.fit(X_train, y_train)\n",
    "\n",
    "# Save model\n",
    "joblib.dump(xgb_model, \"credit_score_model.pkl\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Evaluate Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "y_pred = xgb_model.predict(X_val)\n",
    "print(\"Classification Report (Tuned XGBoost):\\n\")\n",
    "print(classification_report(y_val, y_pred, target_names=label_map.keys()))\n",
    "\n",
    "# Confusion matrix\n",
    "cm = confusion_matrix(y_val, y_pred)\n",
    "disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_map.keys())\n",
    "disp.plot(cmap=\"Blues\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Feature Importance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "xgb.plot_importance(xgb_model, importance_type=\"weight\", max_num_features=10)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. SHAP Explainability"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "explainer = shap.TreeExplainer(xgb_model)\n",
    "shap_values = explainer.shap_values(X_val)\n",
    "\n",
    "# Summary plot\n",
    "shap.summary_plot(shap_values, X_val)\n",
    "\n",
    "# Force plot for first prediction (optional, heavy)\n",
    "# shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_val.iloc[0,:])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Predict on Test Data & Save Submission"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load test set\n",
    "test = pd.read_csv(\"test.csv\")\n",
    "test = test.drop([\"ID\", \"Customer_ID\", \"Name\", \"SSN\"], axis=1)\n",
    "test = test.fillna(test.mean(numeric_only=True))\n",
    "test = pd.get_dummies(test, drop_first=True)\n",
    "test = test.reindex(columns=X.columns, fill_value=0)\n",
    "\n",
    "# Predictions\n",
    "test_preds = xgb_model.predict(test)\n",
    "\n",
    "# Map back to original labels\n",
    "inv_label_map = {v: k for k, v in label_map.items()}\n",
    "test_preds = [inv_label_map[p] for p in test_preds]\n",
    "\n",
    "submission = pd.DataFrame(test_preds, columns=[\"Credit_Score\"])\n",
    "submission.to_csv(\"submission.csv\", index=False)\n",
    "print(\"✅ Predictions saved to submission.csv\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# 📊 Credit Score Prediction Project\n',
    '\n',
    'In this notebook, we will:\n',
    '- Load and clean the data\n',
    '- Train baseline & tuned XGBoost models\n',
    '- Evaluate with classification metrics\n',
    '- Visualize results (confusion matrix, feature importance)\n',
    '- Use SHAP for explainability\n',
    '- Generate predictions for submission']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['# 1. Import Libraries\n',
    'import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'from sklearn.model_selection import train_test_split, RandomizedSearchCV\n',
    'from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay\n',
    'import xgboost as xgb\n',
    'import shap\n',
    'import joblib\n',
    'import warnings\n',
    'warnings.f